In [74]:
# Install and import Gemini SDK
# !pip install -q google-generativeai

import google.generativeai as genai
import json
import re
import os
import ast
from dotenv import load_dotenv
load_dotenv()

True

In [75]:


api_key=os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)
model=genai.GenerativeModel(model_name="models/gemini-2.5-pro")



In [ ]:
#  Define a dummy function (simulating LangChain tools/functions)
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [77]:
# Prompt Gemini to suggest a function call
prompt = """
You are a weather assistant. When asked about the weather, respond with a Python-style function call like:
get_current_weather(location="bangalore jp nagar", unit="celsius")

Now, tell me: What's the weather like in bangalore jp nagar using celsius?
"""

response = model.generate_content(prompt)
print("Gemini Output:\n", response.text)


Gemini Output:
 get_current_weather(location="bangalore jp nagar", unit="celsius")


In [78]:
#  Parse function call from Gemini response
match = re.search(r'get_current_weather\((.*?)\)', response.text)

if match:
    args_str = match.group(1)
    # args = dict(eval(f"dict({args_str})"))  # Convert string to dictionary. it is unsafe to use eval directly
    # safe_args_str = re.sub(r'(\w+)=', r'"\1"=', args_str) # Ensure keys are strings
    # safe_args_str = safe_args_str.replace('=', ':') # Replace '=' with ':' for JSON-like format
    
    
    # import ast
    # args = ast.literal_eval(f"{{{safe_args_str}}}")
    args = dict(re.findall(r'(\w+)="([^"]*)"', args_str))
    
    
    
    print("Parsed Args:", args)

    # Call the function
    result = get_current_weather(**args)
    print("Weather Function Result:", result)
else:
    print("No function call detected.")


Parsed Args: {'location': 'bangalore jp nagar', 'unit': 'celsius'}
Weather Function Result: {"location": "bangalore jp nagar", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [ ]:
# Use Gemini again to summarize the result
summary_prompt = f"The current weather data is: {result}. Please summarize this nicely. write the summary and give the temperature  "
final_response = model.generate_content(summary_prompt)

print("Final User Output:\n", final_response.text)


Final User Output:
 Based on the data provided:

It's a sunny and windy day in Bangalore JP Nagar!

Unfortunately, the specific temperature was not included in the data you provided.


In [80]:
from fpdf import FPDF

# Create PDF instance
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)

# Wrap the response text (multiline)
text = final_response.text
for line in text.split('\n'):
    pdf.multi_cell(0, 10, line)

# Save PDF file
pdf.output("output.pdf")

''